In [1]:
import os
import shutil
from dotenv import load_dotenv
import autogen
import chromadb
from pathlib import Path
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.retrieve_utils  import query_vector_db, create_vector_db_from_dir

load_dotenv()

True

In [2]:
# CREATING VECTOR STORE
class ChromaDBManager:
    def __init__(self, docs_path, db_path, collection_name):
        self.docs_path = Path(docs_path)
        self.db_path = Path(db_path)
        self.collection_name = collection_name

    def setup(self, clear_history=False):
        # Check if the docs directory exists
        if not self.docs_path.exists():
            raise ValueError(f"The docs directory at {self.docs_path} does not exist.")

        # Handle existing database based on clear_history flag
        if self.db_path.exists():
            if clear_history:
                shutil.rmtree(self.db_path)
                print(f"Removed existing directory: {self.db_path}")

        # Create or load the Chroma DB client
        self.client = chromadb.PersistentClient(path=str(self.db_path))
        print(f"Chroma DB client set up at {self.db_path}")

    def create_vector_db(self, max_tokens=3000, chunk_mode="multi_lines"):
        create_vector_db_from_dir(
            dir_path=str(self.docs_path),
            client=self.client,
            collection_name=self.collection_name,
            max_tokens=max_tokens,
            chunk_mode=chunk_mode
        )
        print("Chroma DB created successfully.")

    def query_db(self, query_texts, n_results=10):
        return query_vector_db(
            query_texts=query_texts,
            n_results=n_results,
            client=self.client,
            collection_name=self.collection_name
        )
    
    def get_context(self, results):
        doc_contents = ""
        current_tokens = 0
        _doc_idx = self._doc_idx
        _tmp_retrieve_count = 0
        for idx, doc in enumerate(results["documents"][0]):
            if idx <= _doc_idx:
                continue
            if results["ids"][0][idx] in self._doc_ids:
                continue
            _doc_tokens = self.custom_token_count_function(doc, self._model)
            if _doc_tokens > self._context_max_tokens:
                func_print = f"Skip doc_id {results['ids'][0][idx]} as it is too long to fit in the context."
                print(func_print)
                self._doc_idx = idx
                continue
            if current_tokens + _doc_tokens > self._context_max_tokens:
                break
            func_print = f"Adding doc_id {results['ids'][0][idx]} to context."
            print(func_print)
            # print(func_print, "green"), flush=True)
            current_tokens += _doc_tokens
            doc_contents += doc + "\n"
            self._doc_idx = idx
            self._doc_ids.append(results["ids"][0][idx])
            self._doc_contents.append(doc)
            _tmp_retrieve_count += 1
            if _tmp_retrieve_count >= self.n_results:
                break
        return doc_contents


In [3]:
chroma_db_manager = ChromaDBManager('../data/docs', '../data/chromadb', 'autogen-discord')
chroma_db_manager.setup(clear_history=False)
chroma_db_manager.create_vector_db()

Chroma DB client set up at ../data/chromadb
Chroma DB created successfully.


In [4]:
# CONFIGURATIONS
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../../../.env',
    model_api_key_map={
        "gpt-4-1106-preview": "OPENAI_API_KEY",
    },
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
        }
    }
)

query_vector_db_tool_config = {
    "name": "query_vector_db",
    "description": "Function to query the Chroma vector database.",
    "parameters": {
        "type": "object",
        "properties": {
            "query_texts": {"type": "array", "items": {"type": "string"}},
        },
        "required": ["query_texts"]
    }
}

get_context_tool_config = {
    "name": "get_context",
    "description": "Function to get the context from query results of the Chroma vector database.",
    "parameters": {
        "type": "object",
        "properties": {
            "results": {"type": "object"}
        },
        "required": ["results"]
    }
}


llm_config = {
    "config_list": config_list,
    "assistant_id": None,
    "tools": [
        {
            "type": "function",
            "function": query_vector_db_tool_config
        },
        {
            "type": "function",
            "function": get_context_tool_config
        }
    ],
    "model": "gpt-4-1106-preview"
}

In [10]:
RAG_ASSISTANT_SYSTEM_MESSAGE = """As a Retrieve-Augmented Chatbot, your role involves analyzing message history and using that to inform your responses. 
When a question is presented, you should:
Step 1: Determine the user's intent by considering the question and its context. The intent might involve generating code or answering a question, or both.
Step 2: Formulate a response based on the identified intent.

If the task involves code generation, adhere to the specified coding format. 
For question-answering tasks, aim for concise and precise responses.


For contextual understanding, use the `query_vector_db` tool to access information from the Chroma vector database. 
Enhance your replies by integrating context extracted from the Chroma DB using the `get_context` function. 
This approach will provide a well-informed basis for your responses.
If there is relevant links in the chat logs, provide the fill link in the response.
Say TERMINATE when everything is done and the overall task is complete.
"""

In [11]:
# INSTANTIATE AGENTS

sql_assistant = GPTAssistantAgent(
    name="Chroma_RAG_Assistant",
    instructions=RAG_ASSISTANT_SYSTEM_MESSAGE,
    llm_config=llm_config
)

sql_assistant.register_function(
    function_map={
        "query_vector_db": chroma_db_manager.query_db,
        "get_context": chroma_db_manager.get_context   
    }
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3
)

initial_message = "What is Autogen?"
user_proxy.initiate_chat(sql_assistant, message=initial_message)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
user_proxy (to Chroma_RAG_Assistant):

What is Autogen?

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_vector_db...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(query_vector_db, Sucess: True) : {'ids': [['doc_79', 'doc_68', 'doc_151', 'doc_158', 'doc_20', 'doc_116', 'doc_86', 'doc_104', 'doc_162', 'doc_106']], 'distances': [[0.3410879969596863, 0.35183876752853394, 0.36569702116146097, 0.3802331123892657, 0.4030306935310364, 0.41339688844423306, 0.41603583097457886, 0.4272402840826177, 0.4435499073212885, 0.44422628686505516]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['\nI have worked to load quivr with a manual for a restaurants stove and work through troubleshooting scenarios.\n\nThat is something I am looking to autogen for```.\n\nIn general, at 2023-10-10 01:47:41 a user named yuzhun said ```I tried gpt3.5 for chess play and it failed, wondering how do autogen choose model? Is there a priority flag? Do autogen auto retry with different model in the config file?```.\n\nIn general, at 2023-10-10 01:

Chroma_RAG_Assistant (to user_proxy):

AutoGen appears to be an open-source project associated with Microsoft, designed for working with machine learning models, notably large language models (LLMs). It offers a framework that can help users build applications leveraging these models. Moreover, the project supports the creation of complex applications, such as multi-agent systems, and enables users to manage the dialogue flow in such systems. There are resources like tutorials, documentation, and GitHub repositories for users to learn how to use AutoGen, as indicated in user discussions and references to official AutoGen information sources.

For additional information and official documentation, you may refer to the GitHub page at https://github.com/microsoft/autogen or the documentation site at https://microsoft.github.io/autogen/.

TERMINATE


--------------------------------------------------------------------------------


In [8]:
user_proxy.send(recipient=sql_assistant, message="Explain what the config_list is?")

user_proxy (to Chroma_RAG_Assistant):

Explain what the config_list is?

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_vector_db...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(query_vector_db, Sucess: True) : {'ids': [['doc_147', 'doc_121', 'doc_112', 'doc_126', 'doc_134', 'doc_17', 'doc_5', 'doc_76', 'doc_43', 'doc_55']], 'distances': [[0.5220403791312935, 0.5548239942504392, 0.5996779900457571, 0.65499586374803, 0.6581914901371664, 0.7117836475372314, 0.711988627910614, 0.7250232696533203, 0.7343909740447998, 0.7462843656539917]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['\nconfig_list = autogen.config_list_from_json(\n    "OAI_CONFIG_LIST",\n    filter_dict={\n        "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],\n    },\n)\n\nllm_config={\n    "request_timeout": 600,\n    "seed": 42,\n    "config_list": config_list,\n    "temperature": 0,\n}```.\n\nIn dev-contributors, at 2023-10-02 11:32:29 a user named vincentjedi said ```Back to basic,  how are all the py files under the autogen directo

Chroma_RAG_Assistant (to user_proxy):

The `config_list` mentioned in various discussions seems to be a configuration object related to Autogen, which is likely a framework or tool associated with automating interactions with large language models (LLMs). In the context discussed, the `config_list` appears to be used to specify configurations for agents, particularly specifying which models to use (e.g., GPT-3.5-turbo, GPT-4), as well as other settings such as timeouts, temperature settings for generating responses, etc. 

There are references to the `config_list` being loaded from a JSON file, potentially also involving environment variables or other sources, to set up the configuration for an agent within Autogen. This configuration would then inform how the agent behaves, which model it uses for generating responses, and other aspects of its operation. It seems to be a central part of setting up and controlling the behavior of agents in the Autogen framework.


---------------------

In [9]:
user_proxy.send(recipient=sql_assistant, message="How can you use Chainlit with Autogen" )

user_proxy (to Chroma_RAG_Assistant):

How can you use Chainlit with Autogen

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_vector_db...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(query_vector_db, Sucess: True) : {'ids': [['doc_36', 'doc_181', 'doc_114', 'doc_180', 'doc_159', 'doc_97', 'doc_183', 'doc_164', 'doc_140', 'doc_0'], ['doc_79', 'doc_68', 'doc_151', 'doc_158', 'doc_20', 'doc_116', 'doc_86', 'doc_104', 'doc_162', 'doc_106']], 'distances': [[0.7008378505706787, 0.7394188291629203, 0.7775983947633831, 0.7988503588646156, 0.7994156680043922, 0.8151148557662964, 0.8155384921885493, 0.8166936227950747, 0.8209643318483033, 0.8221719264984131], [0.3410881757736206, 0.35183900594711304, 0.36569699762005214, 0.38023305616252234, 0.4030305743217468, 0.41339678731708907, 0.4160357713699341, 0.4272402956254806, 0.44354990913842995, 0.4442262113759967]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['\nIn general, at 2023-10-31 10:14:12 a user named antoineross said `

Chroma_RAG_Assistant (to user_proxy):

Based on the search results, Chainlit seems to be a tool or library used to create web applications that can interact with Autogen. There is a Medium article mentioned in the retrieved documents describing the creation of a basic web application using Chainlit with Autogen, which suggests that Chainlit can be integrated with Autogen to develop user interfaces.

To use Chainlit with Autogen, one would typically need to understand both Chainlit and Autogen, write code to define the Autogen agents and their interactions, and use Chainlit to build a front-end interface that communicates with these agents. Chainlit would likely handle the user inputs and outputs while Autogen agents perform the tasks based on those inputs.


--------------------------------------------------------------------------------
user_proxy (to Chroma_RAG_Assistant):



--------------------------------------------------------------------------------
Chroma_RAG_Assistant (to use